In [1]:
import numpy as np
import pandas as pd
import urllib.request

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

import tensorflow as tf

import os

# Any results you write to the current directory are saved as output.
from google.colab import drive
drive.mount('/content/gdrive')

Using TensorFlow backend.


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
os.getcwd()

'/content'

In [0]:
import os
os.chdir("/content/gdrive/My Drive/Galvanize Adm/Siddhartha")

In [0]:
text = open('Siddhartha.txt').read()
#text = text[300:].lower()  # removing unnecesary content in the begining

In [5]:
text[:500]

'THE SON OF THE BRAHMAN\n\nIn the shade of the house, in the sunshine of the riverbank near the\nboats, in the shade of the Sal-wood forest, in the shade of the fig tree\nis where Siddhartha grew up, the handsome son of the Brahman, the young\nfalcon, together with his friend Govinda, son of a Brahman.  The sun\ntanned his light shoulders by the banks of the river when bathing,\nperforming the sacred ablutions, the sacred offerings.  In the mango\ngrove, shade poured into his black eyes, when playing as '

In [0]:
characters = sorted(list(set(text)))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

In [7]:
pd.DataFrame(n_to_char, index = ['']).iloc[0:4, 0:4]

,0,1,2,3
,\n,,!,""""


In [23]:
pd.DataFrame(char_to_n, index = ['']).iloc[0:4, 0:4]

,,,!,""""
,0,1,2,3


In [25]:
len(n_to_char)

64

In [0]:
X = []
Y = []
length = len(text)
seq_length = 100
for i in range(0, length-seq_length, 1):
     sequence = text[i:i + seq_length]
     label =text[i + seq_length]
     X.append([char_to_n[char] for char in sequence])
     Y.append(char_to_n[label])

In [10]:
print(X[0][0:10])
print(Y[0:10])

[32, 20, 17, 1, 31, 27, 26, 1, 27, 18]
[46, 51, 1, 57, 45, 42, 1, 56, 45, 38]


In [0]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
Y_modified = np_utils.to_categorical(Y)

In [12]:
len(X), X_modified.shape, Y_modified.shape

(217175, (217175, 100, 1), (217175, 64))

In [13]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# # tpu
# model = tf.keras.models.Sequential()
# model.add(tf.keras.layers.LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.LSTM(400))
# model.add(tf.keras.layers.Dropout(0.2))
# model.add(tf.keras.layers.Dense(Y_modified.shape[1], activation='softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics= ['accuracy'])

In [0]:
# TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
# tf.logging.set_verbosity(tf.logging.INFO)

# tpu_model = tf.contrib.tpu.keras_to_tpu_model(
# model,
# strategy = tf.contrib.tpu.TPUDistributionStrategy(
# tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))

In [16]:
model.summary()
#tpu_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 400)          643200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 400)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 400)               1281600   
_________________________________________________________________
dropout_2 (Dropout)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                25664     
Total params: 1,950,464
Trainable params: 1,950,464
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(X_modified, Y_modified, epochs = 2, batch_size = 100)

model.save_weights('text_generate.h5')

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
217175/217175 [==============================] - 933s 4ms/step - loss: 2.7958
Epoch 2/2
217175/217175 [==============================] - 933s 4ms/step - loss: 2.4765


In [0]:
model.load_weights('text_generate.h5')

In [0]:
string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]
# generating characters
for i in range(400):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose = 0))
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [20]:
string_mapped[0:10]

[1, 56, 38, 38, 55, 42, 42, 51, 44, 1]

In [21]:
full_string[0:10]

[' ', 'i', 'n', ' ', 't', 'h', 'e', ' ', 's', 'h']

In [22]:
#combining text
txt=""
for char in full_string:
    txt = txt + char
txt

' in the shade of the Sal-wood forest, in the shade of the fig tree\nis where Siddhartha grew up, the saaree and the saaree of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saareeng of the saa'

In [0]:
pos_to_n = {char:n for n, char in enumerate(characters)}